In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path

# fmt:off
_ = [sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent / "python"] if str(path) not in sys.path] #fmt:on

In [ ]:
import litellm
from devtools import debug
from langchain_community.llms.edenai import EdenAI
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

from python.ai_core.llm import get_llm

llm = get_llm()


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


structured_llm = llm.with_structured_output(Joke)
debug(structured_llm.invoke("Tell me a joke about cats"))

In [ ]:
from devtools import debug
from langchain_community.chat_models.edenai import ChatEdenAI
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


llm_with_tools = llm.bind_tools([add])

query = "What is 11 + 11?"

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

In [ ]:
from python.ai_core.llm import get_llm

llm_oai = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm_groq = ChatGroq(model="lLama3-70b-8192")
llm_eden = EdenAI(
    feature="text",
    provider="openai",
    model="gpt-3.5-turbo-instruct",
)

llm = get_llm(llm_id="llama3_8_deepinfra")


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

In [ ]:
llm_with_tools = llm.bind_tools([add])
llm_with_tools.invoke("What is 45 +  99 ?").tool_calls  # type: ignore # works

# llm_with_tools = llm_eden.bind_tools([add]).tool_calls   # AttributeError: 'EdenAI' object has no attribute 'bind_tools'
# llm_with_tools = llm_groq.bind_tools([add]).tool_calls   AttributeError: 'ChatGroq' object has no attribute 'tool_calls'

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field


class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


structured_llm = llm_oai.with_structured_output(Joke)
debug(structured_llm.invoke("Tell me a joke about cats"))

In [ ]:
structured_llm = llm_eden.with_structured_output(Joke)

In [7]:
from devtools import debug
from langchain_community.chat_models.edenai import ChatEdenAI
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


llm_with_tools = llm.bind_tools([add])

query = "What is 11 + 11?"

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

r = llm_with_tools.invoke(messages)

debug(r)

/tmp/ipykernel_43442/2384772665.py:34 <module>
    r: AIMessage(
        content='The sum of 11 and 11 is 22.',
        response_metadata={
            'openai': {
                'status': 'success',
                'generated_text': 'The sum of 11 and 11 is 22.',
                'message': [
                    {
                        'role': 'user',
                        'message': 'What is 11 + 11?',
                        'tools': [
                            {
                                'name': 'add',
                                'description': (
                                    'Adds a and b.\n'
                                    '\n'
                                    '    Args:\n'
                                    '        a: first int\n'
                                    '        b: second int'
                                ),
                                'parameters': {
                                    'type': 'object',
                        

AIMessage(content='The sum of 11 and 11 is 22.', response_metadata={'openai': {'status': 'success', 'generated_text': 'The sum of 11 and 11 is 22.', 'message': [{'role': 'user', 'message': 'What is 11 + 11?', 'tools': [{'name': 'add', 'description': 'Adds a and b.\n\n    Args:\n        a: first int\n        b: second int', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}], 'tool_calls': None}, {'role': 'assistant', 'message': 'The sum of 11 and 11 is 22.', 'tools': None, 'tool_calls': []}], 'cost': 6.3e-05}}, id='run-f189e720-798b-4dcc-8bd1-bd0a583bf7cf-0')

In [8]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Search Assistant"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100"})
debug(r)



> Entering new AgentExecutor chain...


/home/tcl/.cache/pypoetry/virtualenvs/genai-training-2X6HL8i2-py3.10/lib/python3.10/site-packages/langchain_community/chat_models/edenai.py:611: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")



Invoking: `add` with `{'a': 12, 'b': 100}`


112

/home/tcl/.cache/pypoetry/virtualenvs/genai-training-2X6HL8i2-py3.10/lib/python3.10/site-packages/langchain_community/chat_models/edenai.py:611: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn("stream: Tool use is not yet supported in streaming mode.")


The sum of 12 and 100 is 112.

> Finished chain.
/tmp/ipykernel_43442/1377134187.py:31 <module>
    r: {
        'input': 'what is 12  + 100',
        'output': 'The sum of 12 and 100 is 112.',
    } (dict) len=2


{'input': 'what is 12  + 100', 'output': 'The sum of 12 and 100 is 112.'}

In [9]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import EdenAI
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

llm = EdenAI(
    feature="text", provider="openai", params={"temperature": 0.2, "max_tokens": 250}
)

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"]),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    return_intermediate_steps=True,
)

/home/tcl/.cache/pypoetry/virtualenvs/genai-training-2X6HL8i2-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [10]:
input_ = """i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""
result = agent_chain(input_)

/home/tcl/.cache/pypoetry/virtualenvs/genai-training-2X6HL8i2-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 We should use edenai_explicit_content_detection_text to check if the text contains explicit content. If it does, we should use edenai_text_to_speech to convert the text into speech.
Action: edenai_explicit_content_detection_text
Action Input: 'i want to slap you'
Observation: nsfw_likelihood: 4
"sexual": 1
"hate": 1
"harassment": 4
"self-harm": 1
"sexual/minors": 1
"hate/threatening": 1
"violence/graphic": 1
"self-harm/intent": 1
"self-harm/instructions": 1
"harassment/threatening": 1
"violence": 4
Thought: The text contains explicit content, specifically related to sexual harassment and violence.
Action: edenai_text_to_speech
Action Input: 'i want to slap you'
Observation: https://d14uq1pz7dzsdq.cloudfront.net/644d5e31-1a6b-4715-9617-9e7f6c1d68d8_.mp3?Expires=1718872722&Signature=TUAfFnp5gg7Ig~AAt0hKI3sXa9hnJ6U1hBrcq5pXpcZg8GsVVZmCLLJmEuDAARoPPTS6eYUbe305sg0J9hVpYZTuK6Q9RmfXvRxmjdyFcHkpaWmfnFW8PUgPk1GjkVceIwaCa-XLi1VvnkzkMubdDzJVYOfqF7mI355f8-v